# Ceneo Scraper

## Struktura pojedynczej opinii
|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|.user-post__author-name|author|
|rekomendacja|.user-post__author-recomendation|recommendation|
|gwiazdki|.user-post__score-count|stars|
|treść|.user-post__text|content|
|lista zalet|.review-feature__title--positives ~ .review-feature__item|pros|
|lista wad|.review-feature__title--negatives ~ .review-feature__item|cons|
|dla ilu przydatna|.vote-yes > span|helpful|
|dla ilu nieprzydatna|.vote-no > span|unhelpful|
|data wystawienia|.user-post__published > time:nth-child(1)|publish_date|
|data zakupu|.user-post__published > time:nth-child(2)|purchase_date|

## Załadowanie bibliotek

In [10]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Funkcja

In [11]:
def extract(ancestor,selektor=None,attribute=None,return_list=False):
    if selektor:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selektor)]
            return [tag.text.strip() for tag in ancestor.select(selektor)]
        if attribute: 
            try:
                return ancestor.select_one(selektor)[attribute].strip()
            except TypeError:
                return None
        try: 
            return ancestor.select_one(selektor).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()
   

## Selektory składowych opinii

In [12]:
selectors = {
        "opinion_id" : (None,"data-entry-id"),
        "author" : (".user-post__author-name",),
        "recommendation" : (".user-post__author-recomendation",),
        "stars" : (".user-post__score-count",),
        "content" : (".user-post__text",),
        "pros" : (".review-feature__title--positives ~ .review-feature__item",None,True),
        "cons" : (".review-feature__title--negatives ~ .review-feature__item",None,True),
        "helpful" : (".vote-yes > span",),
        "unhelpful" : (".vote-no > span",),
        "publish_date" : (".user-post__published > time:nth-child(1)","datetime"),
        "purchase_date" : (".user-post__published > time:nth-child(2)","datetime"),
        }

## Adres URL pierwszej strony z opiniami o produkcie

In [13]:
#product_id = "138331381"
# product_id = "39562616"
product_id = input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

## Pobieranie wszystkich opinii z kodu HTML strony


In [14]:
all_opinions=[]
while(url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key,value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url= "https://www.ceneo.pl/"+page_dom.select_one("a.pagination__next")["href"].strip()
    except TypeError: url= None

## Zapis opinii do pliku

In [15]:
if not os.path.exists("opinions"):
    os.makedirs("opinions")
with open(f"opinions/{product_id}.json","w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)